In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/MyDrive/SPPNet')

In [3]:
!git clone https://github.com/yifanjiang97/sppnet-pytorch.git

Cloning into 'sppnet-pytorch'...
remote: Enumerating objects: 31, done.
remote: Total 31 (delta 0), reused 0 (delta 0), pack-reused 31
Unpacking objects: 100% (31/31), done.


spp_layer


In [3]:
import math
def spatial_pyramid_pool(self, previous_conv, num_sample, previous_conv_size, out_pool_size):
  '''
  previous_conv: a tensor vector of previous convolution layer
  num_sample: an int number of image in the batch
  previous_conv_size: an int vector [height, width] of the matrix features size of previous convolution layer
  out_pool_size: a int vector of expected output size of max pooling layer
  
  returns: a tensor vector with shape [1 x n] is the concentration of multi-level pooling
  '''

  for i in range(len(out_pool_size)):
    h_wid = int(math.ceil(previous_conv_size[0] / out_pool_size[i]))
    w_wid = int(math.ceil(previous_conv_size[1] / out_pool_size[1]))
    # 反卷积计算
    h_pad = (h_wid * out_pool_size[i] - previous_conv_size[0] + 1) / 2
    w_pad = (w_wid * out_pool_size[i] - previous_conv_size[1] + 1) / 2
    maxpool = nn.MaxPool2d((h_wid, w_wid), stride= (h_wid, w_wid), padding = (h_pad, w_pad))
    x = maxpool(previous_conv)
    if(i == 0):
      spp = x.view(num_sample, -1)
    else:
      spp = torch.cat((spp, x.view(num_sample, -1)), 1)
    
  return spp

cnn_with_spp


In [7]:
import torch
import torch.nn as nn
from torch.nn import init
import functools
from torch.autograd import Variable
import numpy as np
import

class SSP_NET(nn.Module):
  '''
  A CNN model which adds spp layer so that we can input multi-size tensor
  '''

  def __init__(self, opt, input_nc, ndf=64, gpu_ids=[]):
    super(SPP_NET, self).__init__()
    super.gpu_ids = gpu_ids
    self.output_num = [4,2,1]

    self.conv1 = nn.Conv2d(input_nc, ndf, 4, 2, 1, bias=False)

    self.conv2 = nn.Conv2d(ndf, ndf * 2, 4, 1, 1, bias=False)
    self.BN1 = nn.BatchNorm2d(ndf * 2)

    self.conv3 = nn.Conv2d(ndf * 2, ndf * 4, 4, 1, 1, bias=False)
    self.BN2 = nn.BatchNorm2d(ndf * 4)

    self.conv4 = nn.Conv2d(ndf * 4, ndf * 8, 4, 1, 1, bias=False)
    self.BN3 = nn.BatchNorm2d(ndf * 8)

    self.conv5 = nn.Conv2d(ndf * 8, 64, 4, 1, 0, bias=False)
    self.fc1 = nn.Linear(10752, 4096)
    self.fc2 = nn.Linear(4096, 1000)
  
  def forward(self, x):
    x = self.conv1(x)
    x = self.LReLU1(x)

    x = self.conv2(x)
    x = F.leaky_relu(self.BN1(x))

    x = self.conv3(x)
    # x = F.leaky_relu(self.BN3(x))
    # x = self.conv5(x)
    spp = spatial_pyramid_pool(x, 1, [int(x.size(2)), int(x.size(3))], self.output_num)
    # print(spp.size())
    fc1 = self.fc1(spp)
    fc2 = self.fc2(fc1)
    s = nn.Sigmoid()
    output = s(fc2)
    return output